#Bibliothèque et téléchargement des données

In [ ]:
import torchvision
from torchvision import datasets
from torchvision import transforms as T
import torch
from torch.utils.data import DataLoader, ConcatDataset, Subset
from collections import Counter
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
import os

print("Chargement des bibliothéques")

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")

print("Path to dataset files:", path)

In [ ]:
contents = os.listdir(os.path.join(path, "chest_xray"))
print("Directory contents:", contents)

In [ ]:
contents = os.listdir(os.path.join(path, "chest_xray/train"))
print("Directory contents:", contents)

#PARTIE 1

In [ ]:
content = ["chest_xray/train", "chest_xray/test", "chest_xray/val"]

for path_ in content:
    dataset = torchvision.datasets.ImageFolder(os.path.join(path, path_))

    classes = dataset.classes
    lenght = len(dataset)
    index = dataset.class_to_idx
    targets = dataset.targets
    stats = Counter(dataset.targets)
    print(f"{path_}\n")

    for i in index.keys():
        for j in stats.keys():
            if index[i] == j:
                print(f"Class: {i}, Number of images: {stats[j]}, percentage: {stats[j]/lenght:.2%}")

    print("Number of images: ", lenght, "\nClasses: ", classes, "\nClass to index mapping: ", index, "\n")

    plt.subplot(1, 2, 1)
    plt.bar(stats.keys(), stats.values())
    plt.xticks(list(index.values()), list(index.keys()))
    plt.xlabel("Classes")
    plt.ylabel("Number of images")
    plt.title(f"Distribution of classes in {path_}")

    plt.subplot(1, 2, 2)
    plt.pie(stats.values(), labels=list(index.keys()), autopct="%1.1f%%")
    plt.show()



In [ ]:
import matplotlib.pyplot as plt
import os
from PIL import Image


db_path = os.path.join(path, 'chest_xray')
train_normal = os.path.join(db_path, 'train', 'NORMAL')
train_pneu = os.path.join(db_path, 'train', 'PNEUMONIA')
image_hasard_sain = os.listdir(train_normal)[0]
image_hasard_pneumo = os.listdir(train_pneu)[0]
chemin_image_sain = os.path.join(train_normal, image_hasard_sain)
chemin_image_pneumo = os.path.join(train_pneu, image_hasard_pneumo)

plt.figure(figsize=(12, 6))

# Image Normale
plt.subplot(1, 2, 1) # 1 ligne, 2 colonnes, position 1
plt.imshow(Image.open(chemin_image_sain), cmap='gray')
plt.title("Poumon NORMAL")

# Image Pneumonie
plt.subplot(1, 2, 2) # 1 ligne, 2 colonnes, position 2
plt.imshow(Image.open(chemin_image_pneumo), cmap='gray')
plt.title("Poumon PNEUMONIE")

plt.show()

In [ ]:
import numpy as np
def print_stats(nom_dossier, db_path):
    liste_moy = []
    liste_ecart_type = []

    fichiers = os.listdir(db_path)

    for image in fichiers :
        chemin_image = os.path.join(db_path, image)
        img = Image.open(chemin_image)
        px = np.array(img)
        liste_moy.append(np.mean(px))
        liste_ecart_type.append(np.std(px))

    return {
        'nom': nom_dossier,
        'mean': np.mean(liste_moy),
        'std': np.mean(liste_ecart_type),
        'count': len(liste_moy)
    }

stats_sain = print_stats("SAIN", train_normal)
stats_pneu = print_stats("PNEUMONIE", train_pneu)

print(stats_sain)
print(stats_pneu)


In [ ]:
import pandas as pd
# On crée une liste avec nos deux dictionnaires de stats
resultats = [stats_sain, stats_pneu]

# Pandas transforme automatiquement une liste de dictionnaires en tableau !
comparaison = pd.DataFrame(resultats)

# On renomme les colonnes pour que ce soit joli
comparaison.columns = ['Classe', 'Moyenne Luminosité', 'Contraste (Std)', 'Nombre Images']

print(comparaison)

In [ ]:
# On convertit les moyennes (0-255) en format PyTorch (0-1)
final_mean = (stats_sain['mean'] + stats_pneu['mean']) / 2 / 255
final_std = (stats_sain['std'] + stats_pneu['std']) / 2 / 255

print(f"Mean pour Normalize: {final_mean:.4f}")
print(f"Std pour Normalize: {final_std:.4f}")

#THEME 2

Les transformers sont réutilisés dans la partie 7

In [ ]:
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
#On redimensionne les images (224×224) et on les normalise
BATCH_SIZE = 128
NUM_WORKERS = 2
transform_train = transforms.Compose([
        transforms.Resize(256),
        transforms.RandomRotation(10), #permet de généraliser
        transforms.RandomResizedCrop(224), #zoom etape par etape
        transforms.ToTensor(), #gagner en temps de calcul
        transforms.Normalize(mean=[final_mean], std=[final_std])
    ])

transform_test = transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[final_mean], std=[final_std])
    ])

print("transform train et test crée")


In [ ]:
import torch
import numpy as np
import random

#Permet que l'expérience soit reproductible
#Sans cela, le hasard change à chaque fois et on ne peut pas comparer les résultats d'un jour à l'autre
def fix_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True # Désactive les algos non-déterministes
    torch.backends.cudnn.benchmark = False     # Évite les variations de performance

fix_seed(42)

In [ ]:
train_dataset = os.path.join(db_path, 'train')
test_dataset = os.path.join(db_path, 'test')
val_dataset = os.path.join(db_path, 'val')

full_train = datasets.ImageFolder(os.path.join(db_path, "train"), transform=transform_train)
train_clean = datasets.ImageFolder(os.path.join(db_path, "train"), transform=transform_test)
full_test = datasets.ImageFolder(os.path.join(db_path, "test"), transform=transform_test)

indices = torch.randperm(len(full_train), generator=torch.Generator().manual_seed(42))
train_idx, val_idx = indices[:int(0.9*len(full_train))], indices[int(0.9*len(full_train)):]

#Au lieu de charger 5000 images d'un coup on les envoies par petits paquets (Batches)
train_loader = DataLoader(Subset(full_train, train_idx), batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(full_test, batch_size = BATCH_SIZE, shuffle=True)
val_loader = DataLoader(ConcatDataset([datasets.ImageFolder(os.path.join(db_path, "val"), transform=transform_test),
                                       Subset(train_clean, val_idx)]), batch_size=BATCH_SIZE)

In [ ]:
import torch

baseline_metrics = {
    'mean': final_mean,
    'std': final_std,
    'classes': ['NORMAL', 'PNEUMONIA'],
    'distribution': [len(os.listdir(train_normal)), len(os.listdir(train_pneu))]
}

torch.save(baseline_metrics, 'pneumonia_baseline_metrics.pt')
print("Baseline sauvegardée pour l'étape 6")

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

#On crée un réseau avec 4 blocs de convolution, de la Batch Normalization et du Dropout.
#Convolution : Le modèle apprend à détecter des formes
#Batch Normalization : Ça stabilise l'apprentissage
#Dropout : On éteint la moitié des neurones au hasard pendant l'entraînement.
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(SimpleCNN, self).__init__()

        # Bloc 1 : 224x224 -> 112x112
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)

        # Bloc 2 : 112x112 -> 56x56
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)

        # Bloc 3 : 56x56 -> 28x28
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)

        # Bloc 4 (Ajouté pour le budget paramètres) : 28x28 -> 14x14
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)

        self.pool = nn.MaxPool2d(2, 2)

        # MLP Final : Entrée 64 * 14 * 14 = 12 544
        self.fc1 = nn.Linear(64 * 14 * 14, 128) # Réduit à 128 pour l'efficacité
        self.fc2 = nn.Linear(128, num_classes)

        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x)))) # 4ème réduction

        x = x.view(x.size(0), -1) # Flatten propre
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

print("Le CNN est réalisé")

SImpleCNN réutilisé à la partie 7

In [ ]:
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter

# Le dataset a sûrement beaucoup plus de cas "Pneumonie" que "Normal".
# Sans ces poids, le modèle choisirait la facilité en prédisant "Pneumonie" tout le temps.
# On lui dit : "Si tu te trompes sur un cas Normal, je te punis 2.8 fois plus fort".
# Adam : C'est un moteur intelligent qui ajuste la vitesse d'apprentissage tout seul.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

weights = torch.tensor([2.8, 1.0]).to(device)
criterion = nn.CrossEntropyLoss(weight=weights)

writer = SummaryWriter('runs/pneumonia_baseline_1')

In [ ]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Nombre de paramètres : {total_params:,}")
# On devrait être autour de 1.2M - 1.5M selon les tailles des FC.

In [ ]:
# Test rapide : Est-ce que les loaders existent ?
try:
    print(f"Train loader : {type(train_loader)} - OK")
    print(f"Val loader : {type(val_loader)} - OK")
    # On vérifie si on peut tirer un batch
    images, labels = next(iter(train_loader))
    print(f"Batch testé : {images.shape}")
except NameError as e:
    print(f" Erreur : La variable n'existe pas. {e}")
except Exception as e:
    print(f"Autre erreur : {e}")

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    # Niveau 1 : Dans la fonction (4 espaces)
    history = {
        'train_loss': [], 'val_loss': [],
        'train_acc': [], 'val_acc': []
    }

    best_val_acc = 0.0
    print("On lance le training")
    for epoch in range(epochs):
        # Niveau 2 : Dans la boucle for (8 espaces)
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        print(f"\nÉpoque {epoch+1}/{epochs}")
        for images, labels in tqdm(train_loader, desc="Training"):
            # Niveau 3 : Dans la boucle des images (12 espaces)
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        # Retour au niveau 2 pour les calculs de fin d'époque
        epoch_train_loss = running_loss / len(train_loader.dataset)
        epoch_train_acc = 100 * correct_train / total_train

        model.eval()
        running_val_loss = 0.0
        correct_val = 0
        total_val = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                running_val_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs.data, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

        epoch_val_loss = running_val_loss / len(val_loader.dataset)
        epoch_val_acc = 100 * correct_val / total_val

        # Mise à jour du dictionnaire history
        history['train_loss'].append(epoch_train_loss)
        history['train_acc'].append(epoch_train_acc)
        history['val_loss'].append(epoch_val_loss)
        history['val_acc'].append(epoch_val_acc)

        print(f"Train Loss: {epoch_train_loss:.4f} | Train Acc: {epoch_train_acc:.2f}%")
        print(f"Val Loss: {epoch_val_loss:.4f} | Val Acc: {epoch_val_acc:.2f}%")

        if epoch_val_acc > best_val_acc:
            best_val_acc = epoch_val_acc
            torch.save(model.state_dict(), 'best_model.pth')
            print(f"⭐ Nouveau meilleur modèle sauvegardé ! (Acc: {best_val_acc:.2f}%)")

    # Retour au niveau 1 : Fin de la fonction
    writer.close()
    print("\nEntraînement terminé !")
    return history

In [ ]:
history_CNN = train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10)

# 2. On affiche les courbes avec Matplotlib
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

# Graphique de la Loss
plt.subplot(1, 2, 1)
plt.plot(history_CNN['train_loss'], label='Train Loss')
plt.plot(history_CNN['val_loss'], label='Val Loss')
plt.title('Courbe de Perte (Loss)')
plt.legend()

# Graphique de l'Accuracy
plt.subplot(1, 2, 2)
plt.plot(history_CNN['train_acc'], label='Train Acc')
plt.plot(history_CNN['val_acc'], label='Val Acc')
plt.title('Courbe de Précision (Accuracy)')
plt.legend()

plt.show()

In [ ]:
def test_final(model, test_loader):
    # Charger les meilleurs poids sauvegardés
    # Pour voir les "Learning Curves". Si la courbe de Validation s'envole alors que le Train descend = KO
    model.load_state_dict(torch.load('best_model.pth'))
    model.eval()

    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Précision Finale sur le Test Set : {100 * correct / total:.2f}%")

# Re-initialize test_loader correctly using the full_test dataset
test_loader = DataLoader(full_test, batch_size=BATCH_SIZE, shuffle=False)

test_final(model, test_loader)

In [ ]:
# En sauvegardant le meilleur score, on garde le "meilleur de l'intelligence" du modèle, même si l'epoch d'après il devient nul.

checkpoint = {
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'epoch': 10,
    'loss': 0.33
}
torch.save(checkpoint, 'full_checkpoint.pth')

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

#On lance une évaluation sur le Test Set
def plot_confusion_matrix(model, test_loader):
    model.load_state_dict(torch.load('best_model.pth'))
    model.eval()

    y_true = []
    y_pred = []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['NORMAL', 'PNEUMONIA'],
                yticklabels=['NORMAL', 'PNEUMONIA'])
    plt.xlabel('Prédiction du Modèle')
    plt.ylabel('Vraie Étiquette')
    plt.title('Matrice de Confusion')
    plt.show()

    print(classification_report(y_true, y_pred, target_names=['NORMAL', 'PNEUMONIA']))


plot_confusion_matrix(model, test_loader)

In [ ]:
def lr_finder(model, train_loader, criterion, optimizer, device):
    lrs = []
    losses = []
    lr = 1e-6

    print("Début du test de Learning Rate...")

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.param_groups[0]['lr'] = lr

        outputs = model(images)
        loss = criterion(outputs, labels)

        # Sécurité : Si la loss explose (devient 4x plus grande que le début), on arrête
        if len(losses) > 0 and loss.item() > losses[0] * 4:
            print(f"Arrêt précoce : La loss explose à LR = {lr:.1e}")
            break

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        lrs.append(lr)
        losses.append(loss.item())
        lr *= 1.1
        if lr > 1: break

    # --- Bloc de Visualisation ---
    if len(lrs) > 0:
        plt.figure(figsize=(10, 6))
        plt.plot(lrs, losses)
        plt.xscale('log')
        plt.xlabel("Learning Rate (échelle log)")
        plt.ylabel("Loss")
        plt.title("LR Finder : Cherchez la pente la plus raide")
        plt.grid(True, which="both", ls="-", alpha=0.5)
        plt.show() # L'affichage final
    else:
        print("Erreur : Aucun point à afficher. Vérifiez votre loader.")



In [ ]:
import torchvision.models as models
import torch.nn as nn

model_resnet = models.resnet18(weights=None).to(device)
model_resnet.fc = nn.Linear(model_resnet.fc.in_features, 2).to(device)

optimizer_test = torch.optim.Adam(model_resnet.parameters(), lr=1e-6)
criterion = nn.CrossEntropyLoss(weight=weights)
model_resnet = model_resnet.to(device)

lr_finder(model_resnet, train_loader, criterion, optimizer_test, device)

In [ ]:
# 1. RÉINITIALISATION
model_resnet = models.resnet18(weights=None).to(device)
model_resnet.fc = nn.Linear(model_resnet.fc.in_features, 2).to(device)

# 2. CONFIGURATION OPTIMISÉE
# On utilise le LR trouvé : 1e-5
optimizer = torch.optim.Adam(model_resnet.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss(weight=weights) # Poids de classe toujours actifs

# 3. LANÇER L'ENTRAÎNEMENT
history_resnet = train_model(model_resnet, train_loader, val_loader, criterion, optimizer, epochs=15)

In [ ]:
import matplotlib.pyplot as plt

# S'assurer que les lignes commencent bien au début (colonne 0)
plt.figure(figsize=(12, 5))

# --- Graphique de la Loss ---
plt.subplot(1, 2, 1)
# On vérifie si history_simple existe avant de tracer
if 'history' in locals():
    plt.plot(history['train_loss'], label='SimpleCNN Train', linestyle='--', color='blue')
if 'history_resnet' in locals():
    plt.plot(history_resnet['train_loss'], label='ResNet18 Train', color='red')
plt.title('Comparaison de la Perte (Loss)')
plt.xlabel('Époques')
plt.ylabel('Loss')
plt.legend()

# --- Graphique de l'Accuracy ---
plt.subplot(1, 2, 2)
if 'history' in locals():
    plt.plot(history['val_acc'], label='SimpleCNN Val', linestyle='--', color='blue')
if 'history_resnet' in locals():
    plt.plot(history_resnet['val_acc'], label='ResNet18 Val', color='red')
plt.title('Comparaison de la Précision (Accuracy)')
plt.xlabel('Époques')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.tight_layout()
plt.show()

#PARTIE 4

In [ ]:
path_onnx = "/kaggle/input/models/thastraudo/model-onnx/onnx/default/1"
if path_onnx :
    print("Accès à ONNX")

In [ ]:
import onnxruntime as ort
import numpy as np

model_path = os.path.join(path_onnx, "model_optimized.onnx")

try:
    session = ort.InferenceSession(model_path)
    input_name = session.get_inputs()[0].name
    print("Succès : Le modèle ONNX est chargé et la session est prête !")
except Exception as e:
    print(f"Erreur lors du chargement : {e}")

In [ ]:
import csv
from datetime import datetime
#Inference logger
#__init__ : C'est la préparation. On crée le fichier production_logs.csv
#on écrit les titres des colonnes : timestamp, la classe trouvée, la confiance du modèle et le temps de calcul
#log : C'est l'action d'écrire. À chaque fois qu'on appelle cette fonction, elle ouvre le fichier, ajoute une ligne à la fin
#(mode 'a' pour "append"), et le referme.

class InferenceLogger:
    def __init__(self, log_file='production_logs.csv'):
        self.log_file = log_file
        # On crée le fichier et on écrit les titres des colonnes
        with open(log_file, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['timestamp', 'pred_class', 'confidence', 'latency_ms'])

    def log(self, pred_class, confidence, latency_ms):
        # On ajoute une ligne à chaque fois qu'on fait une prédiction
        with open(self.log_file, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([datetime.now().isoformat(), pred_class, confidence, latency_ms])

# On crée l'objet logger
logger = InferenceLogger()
print("code run")

In [ ]:
#Simulation de la vraie vie
#On déclenche un chronomètre juste avant que l'image entre dans le modèle.
#On calcule le temps en millisecondes (ms)
#Le modèle donne des scores bruts, Le Softmax transforme ces chiffres en pourcentages
#On envoie toutes ces infos (prédiction, confiance, temps) vers notre carnet de bord créé à l'étape 1.

import time
import numpy as np

production_preds_onnx = []

print("Lancement de l'inférence optimisée (ONNX)...")

for images, labels in test_loader:
    # --- AJUSTEMENT DES DIMENSIONS ---
    # Si l'image est en RGB (3 canaux) mais que ONNX veut du Gris (1 canal)
    if images.shape[1] == 3:
        # On calcule la moyenne des 3 canaux pour simuler du gris
        # Shape passe de [Batch, 3, H, W] à [Batch, 1, H, W]
        images_onnx = images.mean(dim=1, keepdim=True)
    else:
        images_onnx = images

    # 1. Conversion du Tensor PyTorch en tableau NumPy
    input_data = images_onnx.cpu().numpy()

    # 2. Mesure de la latence
    start = time.time()
    # On lance l'inférence sur le moteur ONNX
    outputs = session.run(None, {input_name: input_data})[0]
    latency = (time.time() - start) * 1000 / len(images)

    # 3. Conversion en probabilités (Softmax manuel)
    exp_outputs = np.exp(outputs - np.max(outputs, axis=1, keepdims=True))
    probs = exp_outputs / np.sum(exp_outputs, axis=1, keepdims=True)

    # 4. Récupération des prédictions et confiances
    preds = np.argmax(probs, axis=1)
    confs = np.max(probs, axis=1)

    # 5. Logging
    for pred, conf in zip(preds, confs):
        logger.log(int(pred), float(conf), latency)
        production_preds_onnx.append({'pred': int(pred), 'conf': float(conf), 'latency': latency})

print(f" {len(production_preds_onnx)} prédictions effectuées avec ONNX.")



In [ ]:
#on crée le tableau de bord
df_prod = pd.read_csv('production_logs.csv')

# Calcul des KPI (Indicateurs Clés de Performance)
avg_confidence = df_prod['confidence'].mean()
latency_p95 = df_prod['latency_ms'].quantile(0.95) # Le "pire" cas pour 95% des requêtes
pred_distribution = df_prod['pred_class'].value_counts()

# Envoi vers TensorBoard
writer.add_scalar('Production/Avg_Confidence', avg_confidence, 0)
writer.add_scalar('Production/Latency_P95', latency_p95, 0)
writer.add_histogram('Production/Confidences', df_prod['confidence'].values, 0)
writer.close()

print(f"Simulation terminée : {len(production_preds_onnx)} prédictions loguées.")
print(f"Confiance moyenne : {avg_confidence:.3f}")
print(f"Latence P95 : {latency_p95:.2f} ms")

In [ ]:
# Sauvegarde de la Baseline
baseline_production = {
    'avg_confidence': avg_confidence,
    'latency_p95': latency_p95,
    'pred_distribution': dict(pred_distribution)
}
torch.save(baseline_production, 'baseline_production_metrics.pt')
